# Explore here

In [2]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.metrics import accuracy_score, recall_score, f1_score
import joblib

# 1. CARGA DE DATOS

df = pd.read_csv("/workspaces/kNEAREST_NEIGHBORS/data/raw/tmdb_5000_movies.csv")


df.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

In [4]:
d_f = pd.read_csv("/workspaces/kNEAREST_NEIGHBORS/data/raw/tmdb_5000_credits.csv")


d_f.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [5]:
import pandas as pd
import sqlite3

# Ya tienes los DataFrames 'df' y 'd_f' cargados
# Crea una conexión a una base de datos SQLite (se creará si no existe)
conn = sqlite3.connect('movies_database.db')

# Guarda los DataFrames en la base de datos como tablas
df.to_sql('movies', conn, if_exists='replace', index=False)
d_f.to_sql('credits', conn, if_exists='replace', index=False)

# Cierra la conexión para asegurar que se guarden los cambios
conn.close()

In [6]:
import pandas as pd
import sqlite3

# Vuelve a abrir la conexión a la base de datos
conn = sqlite3.connect('movies_database.db')

# Escribe una consulta SQL para unir las tablas 'movies' y 'credits' en base al campo 'title'
query = """
SELECT movies.*, credits.*
FROM movies
INNER JOIN credits ON movies.title = credits.title
"""

# Ejecuta la consulta y guarda el resultado en un nuevo DataFrame
df_merged = pd.read_sql_query(query, conn)

# Cierra la conexión
conn.close()

# Selecciona solo las columnas deseadas
columns_to_keep = ['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']
df_cleaned = df_merged[columns_to_keep]

# Verifica la información del nuevo DataFrame
df_cleaned.info()

# Muestra las primeras filas del DataFrame limpio
print(df_cleaned.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4809 entries, 0 to 4808
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   movie_id  4809 non-null   int64 
 1   title     4809 non-null   object
 2   title     4809 non-null   object
 3   overview  4806 non-null   object
 4   genres    4809 non-null   object
 5   keywords  4809 non-null   object
 6   cast      4809 non-null   object
 7   crew      4809 non-null   object
dtypes: int64(1), object(7)
memory usage: 300.7+ KB
   movie_id                                     title  \
0     19995                                    Avatar   
1       285  Pirates of the Caribbean: At World's End   
2    206647                                   Spectre   
3     49026                     The Dark Knight Rises   
4     49529                               John Carter   

                                      title  \
0                                    Avatar   
1  Pirates of the Caribbe

In [7]:
# Elimina la columna duplicada 'title'
# Primero, identifica cuál de las dos columnas 'title' es la duplicada.
# En este caso, se asume que la segunda columna 'title' es la duplicada.
df_cleaned = df_cleaned.iloc[:, df_cleaned.columns.duplicated(keep='first') == False]

# Verifica la información del nuevo DataFrame después de eliminar la columna duplicada
df_cleaned.info()

# Muestra las primeras filas del DataFrame limpio
print(df_cleaned.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4809 entries, 0 to 4808
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   movie_id  4809 non-null   int64 
 1   title     4809 non-null   object
 2   overview  4806 non-null   object
 3   genres    4809 non-null   object
 4   keywords  4809 non-null   object
 5   cast      4809 non-null   object
 6   crew      4809 non-null   object
dtypes: int64(1), object(6)
memory usage: 263.1+ KB
   movie_id                                     title  \
0     19995                                    Avatar   
1       285  Pirates of the Caribbean: At World's End   
2    206647                                   Spectre   
3     49026                     The Dark Knight Rises   
4     49529                               John Carter   

                                            overview  \
0  In the 22nd century, a paraplegic Marine is di...   
1  Captain Barbossa, long believed to be dead

In [8]:
import pickle
import pandas as pd

# Supongamos que df_cleaned es tu DataFrame
# df_cleaned = pd.DataFrame(...)  # Tu DataFrame ya está definido

# Guardar el DataFrame en un archivo Pickle
with open('df_cleaned.pkl', 'wb') as pickle_file:
    pickle.dump(df_cleaned, pickle_file)

In [9]:
import pickle

# Cargar el DataFrame desde un archivo Pickle
with open('df_cleaned.pkl', 'rb') as pickle_file:
    df_cleaned_loaded = pickle.load(pickle_file)

# Verificar que se haya cargado correctamente
print(df_cleaned_loaded.head())

   movie_id                                     title  \
0     19995                                    Avatar   
1       285  Pirates of the Caribbean: At World's End   
2    206647                                   Spectre   
3     49026                     The Dark Knight Rises   
4     49529                               John Carter   

                                            overview  \
0  In the 22nd century, a paraplegic Marine is di...   
1  Captain Barbossa, long believed to be dead, ha...   
2  A cryptic message from Bond’s past sends him o...   
3  Following the death of District Attorney Harve...   
4  John Carter is a war-weary, former military ca...   

                                              genres  \
0  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
1  [{"id": 12, "name": "Adventure"}, {"id": 14, "...   
2  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
3  [{"id": 28, "name": "Action"}, {"id": 80, "nam...   
4  [{"id": 28, "name": "Action"}, {"id":

In [10]:
# Verificar el tipo de datos en cada columna
print(df_cleaned_loaded.dtypes)

# Verificar filas con datos no válidos en la columna 'genres'
invalid_rows = df_cleaned[df_cleaned['genres'].apply(lambda x: not isinstance(x, list))]
print(invalid_rows)

# Hacer lo mismo para las otras columnas si es necesario

movie_id     int64
title       object
overview    object
genres      object
keywords    object
cast        object
crew        object
dtype: object
      movie_id                                     title  \
0        19995                                    Avatar   
1          285  Pirates of the Caribbean: At World's End   
2       206647                                   Spectre   
3        49026                     The Dark Knight Rises   
4        49529                               John Carter   
...        ...                                       ...   
4804      9367                               El Mariachi   
4805     72766                                 Newlyweds   
4806    231617                 Signed, Sealed, Delivered   
4807    126186                          Shanghai Calling   
4808     25975                         My Date with Drew   

                                               overview  \
0     In the 22nd century, a paraplegic Marine is di...   
1     Captain 

In [11]:
df_cleaned_loaded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4809 entries, 0 to 4808
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   movie_id  4809 non-null   int64 
 1   title     4809 non-null   object
 2   overview  4806 non-null   object
 3   genres    4809 non-null   object
 4   keywords  4809 non-null   object
 5   cast      4809 non-null   object
 6   crew      4809 non-null   object
dtypes: int64(1), object(6)
memory usage: 263.1+ KB


In [12]:
import pandas as pd
import ast

# Supongamos que df_cleaned_loaded ya está definido y tiene las columnas duplicadas
# Primero, eliminamos las columnas duplicadas
df_cleaned_loaded = df_cleaned_loaded.loc[:, ~df_cleaned_loaded.columns.duplicated()]

# Función para convertir cadenas JSON a listas de diccionarios
def safe_eval(val):
    try:
        return ast.literal_eval(val)
    except (ValueError, SyntaxError):
        return []

# Convertir las columnas a listas de diccionarios
df_cleaned_loaded['genres'] = df_cleaned_loaded['genres'].apply(safe_eval)
df_cleaned_loaded['keywords'] = df_cleaned_loaded['keywords'].apply(safe_eval)
df_cleaned_loaded['cast'] = df_cleaned_loaded['cast'].apply(safe_eval)
df_cleaned_loaded['crew'] = df_cleaned_loaded['crew'].apply(safe_eval)

# Función para extraer nombres de géneros o palabras clave
def extract_names(json_list):
    if isinstance(json_list, list):
        names = [item['name'] for item in json_list if isinstance(item, dict) and 'name' in item]
        return names
    return []

# Función para extraer los tres primeros nombres del elenco
def extract_top3_names(json_list):
    if isinstance(json_list, list):
        top3_names = [item['name'] for item in json_list[:3] if isinstance(item, dict) and 'name' in item]
        return top3_names
    return []

# Función para extraer el nombre del director
def extract_director(json_list):
    if isinstance(json_list, list):
        for item in json_list:
            if isinstance(item, dict) and item.get('job') == 'Director':
                return item.get('name', '')
    return ''

# Función para convertir una cadena en una lista de palabras
def convert_to_list(text):
    if isinstance(text, str):
        return text.split()
    return []

# Función para eliminar espacios entre palabras
def remove_spaces(text):
    if isinstance(text, list):
        return [''.join(word.split()) for word in text]
    return []

# Aplicar las transformaciones a las columnas correspondientes
df_cleaned_loaded['genres'] = df_cleaned_loaded['genres'].apply(extract_names)
df_cleaned_loaded['keywords'] = df_cleaned_loaded['keywords'].apply(extract_names)
df_cleaned_loaded['cast'] = df_cleaned_loaded['cast'].apply(extract_top3_names)
df_cleaned_loaded['crew'] = df_cleaned_loaded['crew'].apply(extract_director)
df_cleaned_loaded['overview'] = df_cleaned_loaded['overview'].apply(convert_to_list)

# Eliminar espacios entre palabras en las columnas 'genres', 'cast', 'crew' y 'keywords'
df_cleaned_loaded['genres'] = df_cleaned_loaded['genres'].apply(remove_spaces)
df_cleaned_loaded['cast'] = df_cleaned_loaded['cast'].apply(remove_spaces)
df_cleaned_loaded['crew'] = df_cleaned_loaded['crew'].apply(remove_spaces)
df_cleaned_loaded['keywords'] = df_cleaned_loaded['keywords'].apply(remove_spaces)

# Renombrar el DataFrame como df_done
df_done = df_cleaned_loaded

# Guardar df_done en un archivo Pickle
df_done.to_pickle('df_done.pkl')

# Verificar los cambios
print(df_done.head())

   movie_id                                     title  \
0     19995                                    Avatar   
1       285  Pirates of the Caribbean: At World's End   
2    206647                                   Spectre   
3     49026                     The Dark Knight Rises   
4     49529                               John Carter   

                                            overview  \
0  [In, the, 22nd, century,, a, paraplegic, Marin...   
1  [Captain, Barbossa,, long, believed, to, be, d...   
2  [A, cryptic, message, from, Bond’s, past, send...   
3  [Following, the, death, of, District, Attorney...   
4  [John, Carter, is, a, war-weary,, former, mili...   

                                         genres  \
0  [Action, Adventure, Fantasy, ScienceFiction]   
1                  [Adventure, Fantasy, Action]   
2                    [Action, Adventure, Crime]   
3              [Action, Crime, Drama, Thriller]   
4           [Action, Adventure, ScienceFiction]   

             

In [13]:
import pandas as pd

# Supongamos que df_done ya está definido y tiene las columnas transformadas
df_done = pd.read_pickle('df_done.pkl')  # Si necesitas cargar el DataFrame desde un archivo Pickle

# Combinar las columnas en una sola columna llamada 'tags'
df_done['tags'] = df_done['genres'] + df_done['keywords'] + df_done['cast'] + df_done['crew'] + df_done['overview']

# Convertir la lista de tags en una cadena de texto separada por espacios
df_done['tags'] = df_done['tags'].apply(lambda x: ' '.join(x))

# Verificar el resultado
print(df_done['tags'][0])

# Guardar los cambios en el archivo df_done.pkl
df_done.to_pickle('df_done.pkl')

Action Adventure Fantasy ScienceFiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d SamWorthington ZoeSaldana SigourneyWeaver In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.


In [14]:
print(df_done.columns)

Index(['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew',
       'tags'],
      dtype='object')


In [15]:
import pandas as pd

# Supongamos que df_done ya está definido y tiene las columnas transformadas
df_done = pd.read_pickle('df_done.pkl')  # Si necesitas cargar el DataFrame desde un archivo Pickle

# Verificar las columnas antes de crear 'tags'
print("Columnas antes de crear 'tags':", df_done.columns)

# Combinar las columnas en una sola columna llamada 'tags'
df_done['tags'] = df_done['genres'] + df_done['keywords'] + df_done['cast'] + df_done['crew'] + df_done['overview']

# Convertir la lista de tags en una cadena de texto separada por espacios
df_done['tags'] = df_done['tags'].apply(lambda x: ' '.join(x))

# Verificar las columnas después de crear 'tags'
print("Columnas después de crear 'tags':", df_done.columns)

# Verificar el resultado
print("Primer valor de 'tags':", df_done['tags'][0])

Columnas antes de crear 'tags': Index(['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew',
       'tags'],
      dtype='object')
Columnas después de crear 'tags': Index(['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew',
       'tags'],
      dtype='object')
Primer valor de 'tags': Action Adventure Fantasy ScienceFiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d SamWorthington ZoeSaldana SigourneyWeaver In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.


In [16]:
# import pandas as pd
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.metrics.pairwise import cosine_similarity

# # Cargar el DataFrame desde el archivo Pickle
# df_done = pd.read_pickle('df_done.pkl')

# # Vectorizar el texto en la columna 'tags'
# tfidf = TfidfVectorizer(stop_words='english')
# vectors = tfidf.fit_transform(df_done['tags'])

# # Calcular la similitud del coseno entre los vectores
# similarity = cosine_similarity(vectors)

# # Guardar el DataFrame con los vectores y la matriz de similitud si es necesario
# df_done.to_pickle('df_done_with_vectors.pkl')
# pd.DataFrame(similarity).to_pickle('similarity_matrix.pkl')

# # Verificar el resultado
# print(similarity.shape)  # Debería ser (n_samples, n_samples)
# print(similarity[0])     # Mostrar la similitud del primer elemento con todos los demás

# # Función de recomendación basada en la similitud del coseno
# def recommend(movie):
#     # Asegurarse de que la película esté en el DataFrame
#     if movie not in df_done['title'].values:
#         print(f"La película '{movie}' no está en la base de datos.")
#         return
    
#     # Obtener el índice de la película
#     movie_index = df_done[df_done["title"] == movie].index[0]
    
#     # Calcular las distancias de similitud con todas las demás películas
#     distances = similarity[movie_index]
    
#     # Ordenar las películas por similitud en orden descendente y seleccionar las 5 más similares (excluyendo la propia película)
#     movie_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6]
    
#     # Imprimir los títulos de las 5 películas más similares
#     for i in movie_list:
#         print(df_done.iloc[i[0]].title)

# # Ejemplo de uso
# recommend("Inception")

In [17]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from pickle import dump

# Cargar el DataFrame desde el archivo Pickle
df_done = pd.read_pickle('df_done.pkl')

# Vectorizar el texto en la columna 'tags'
vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(df_done['tags'])

# Crear y entrenar el modelo KNN
model = NearestNeighbors(n_neighbors=6, algorithm='brute', metric='cosine')
model.fit(tfidf_matrix)

# Función para obtener recomendaciones de películas
def get_movie_recommendations(movie_title):
    if movie_title not in df_done['title'].values:
        print(f"La película '{movie_title}' no está en la base de datos.")
        return []
    
    movie_index = df_done[df_done["title"] == movie_title].index[0]
    distances, indices = model.kneighbors(tfidf_matrix[movie_index])
    similar_movies = [(df_done["title"][i], distances[0][j]) for j, i in enumerate(indices[0])]
    return similar_movies[1:]

# Ejemplo de uso
input_movie = "Inception"
recommendations = get_movie_recommendations(input_movie)
print("Film recommendations for '{}'".format(input_movie))
for movie, distance in recommendations:
    print("- Film: {}".format(movie))

# Guardar el modelo
dump(model, open("/workspaces/kNEAREST_NEIGHBORS/models/knn_neighbors-6_algorithm-brute_metric-cosine.sav", "wb"))

Film recommendations for 'Inception'
- Film: Blood and Wine
- Film: Duplex
- Film: The Helix... Loaded
- Film: Cypher
- Film: Hesher
